# Get tweets via API and create Data Frame
Authors: [Brenda Alexandra de Souza Silva](https://github.com/breudes), [Paulo Vanzolini Moura da Silva](https://github.com/paulovanzo) and [Levir Chianca Medeiros dos Santos](https://github.com/levirchianca)

# Imports

In [ ]:
!pip install nxviz==0.6.3
!pip install matplotlib==3.1.3
!python -m pip install scipy
!pip install pandas --upgrade
!pip install Twython

     |████████████████████████████████| 11.2 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 63.5 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 13.1 MB 4.6 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.1
    Uninstalling matplotlib-3.5.1:
      Successfully uninstalled matplotlib-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nxviz 0.6.3 requires matplotlib>=3.3.3, but you have matplotlib 3.1.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import nxviz as nv
import pandas as pd
import numpy as np
import json
import seaborn as sns
import scipy.stats as st
from twython import Twython, TwythonError
from time import sleep

# Get Twitter's API Authentication


> Get crendencials from .txt file (for security purpose)




In [ ]:
# open the keys file
my_file = open("auth_keys.txt", "r")
# read the raw data
content = my_file.read()
# split all lines by  newline character
API_KEY, API_SECRET_KEY, AUTH_TOKEN, AUTH_SECRET_TOKEN, BEARER_TOKEN = content.split("\n")
# close the file
my_file.close()



> Create twython object with API's authentication



In [ ]:
# twython object
twitter = Twython(API_KEY, API_SECRET_KEY)
# get authentication tokens
authentication_tokens = twitter.get_authentication_tokens()
# result
print(authentication_tokens['auth_url'])

https://api.twitter.com/oauth/authenticate?oauth_token=BvF34QAAAAABZQCHAAABfwuoBA8


# Get Tweets with Specific Keywords

This project we work on tweets about **vaccine passport** created on **language portuguese**, which means, tweets all over countries who talks and writes in portuguese - considering this, those keywords will be define on this theme. We are only interested on **recent tweets**, made within the **time of seven days (one week)**. 

In [ ]:
# keywords based on theme 'vaccine passport'
keywords = ['passaporte vacinal', 'passaporte', 'vacina', 'covid-19']
# twitter dict with info we needed
twitter_dict = {'keyword': [], 'user': [], 'tweet_date': [], 'text': [], 'favorite_count': [], 'retweet_count': [], 'user_tweets_count': [], 'user_creation': [], 'retweeted': [], 'retweeted_from': [], 'location': [], 'retweet_fav_count': [], 'retweet_rt_count': []}

# maximum requests available for each keyword defined - there will be a sleep method to restart those resquests within 15 minutes
for keyword in keywords:
  print(keyword)
  for i in range(180): # requests rate from get_twitter_search
    tweets = twitter.search(count=100, q=keyword, result_type='recent', until='2022-02-16', lang='pt')
    
    # get each tweet and add it on dict
    for tweet in tweets['statuses']:
      twitter_dict['keyword'].append(keyword)
      twitter_dict['user'].append(tweet['user']['screen_name'])
      twitter_dict['tweet_date'].append(tweet['created_at'])
      twitter_dict['text'].append(tweet['text'])
      twitter_dict['favorite_count'].append(tweet['favorite_count'])
      twitter_dict['user_tweets_count'].append(tweet['user']['statuses_count'])
      twitter_dict['user_creation'].append(tweet['user']['created_at'])
      twitter_dict['location'].append(tweet['user']['location'])
      twitter_dict['retweet_count'].append(tweet['retweet_count'])
      
      # check if tweet is a retweet or not
      if 'retweeted_status' in tweet:
        twitter_dict['retweeted'].append(True)
        twitter_dict['retweet_fav_count'].append(tweet['retweeted_status']['favorite_count'])
        twitter_dict['retweeted_from'].append(tweet['retweeted_status']['user']['screen_name'])
      else:
        twitter_dict['retweeted'].append(False)
        twitter_dict['retweet_fav_count'].append('')
        twitter_dict['retweeted_from'].append('')
  if(keyword != keywords[3]):
    sleep(901) # 15 minutes in seconds

passaporte vacinal
passaporte
vacina
covid-19


Turn dict into a DataFrame and save as .csv file

In [ ]:
# turn dict into a DF object
df_original = pd.DataFrame({
                   'keyword': twitter_dict['keyword'],
                   'user': twitter_dict['user'],
                   'tweet_date': twitter_dict['tweet_date'],
                   'text': twitter_dict['text'],
                   'favorite_count': twitter_dict['favorite_count'],
                   'retweet_count': twitter_dict['retweet_count'],
                   'user_tweets_count': twitter_dict['user_tweets_count'],
                   'user_creation': twitter_dict['user_creation'],
                   'location': twitter_dict['location'],
                   'retweeted': twitter_dict['retweeted'],
                   'retweet_fav_count': twitter_dict['retweet_fav_count'],
                   'retweeted_from': twitter_dict['retweeted_from'],
})

df_original.drop_duplicates(keep='last') # drop duplicates and keep last occurance
df_original.to_csv("tweets_dataframe.csv") # saves DF on a .csv file

In [ ]:
df_original.head(5)

,keyword,user,tweet_date,text,favorite_count,retweet_count,user_tweets_count,user_creation,location,retweeted,retweet_fav_count,retweeted_from
0,passaporte vacinal,DebacoAlan,Tue Feb 15 23:59:57 +0000 2022,RT @Jrodrigues1954: REJEITOU:\nA Assembleia Le...,0,1387,4577,Fri Sep 29 10:02:12 +0000 2017,"Centro, Novo Hamburgo",True,6351,Jrodrigues1954
1,passaporte vacinal,LigiaGuerra7,Tue Feb 15 23:59:54 +0000 2022,RT @Jrodrigues1954: REJEITOU:\nA Assembleia Le...,0,1387,89906,Sat Feb 03 15:05:08 +0000 2018,"São Paulo, Brasil",True,6351,Jrodrigues1954
2,passaporte vacinal,VilmaThomasCos1,Tue Feb 15 23:59:36 +0000 2022,RT @Jrodrigues1954: REJEITOU:\nA Assembleia Le...,0,1387,70737,Wed Feb 26 23:40:26 +0000 2020,,True,6351,Jrodrigues1954
3,passaporte vacinal,escoladeprofeta,Tue Feb 15 23:59:32 +0000 2022,RT @Jrodrigues1954: REJEITOU:\nA Assembleia Le...,0,1387,17705,Mon Aug 17 17:50:43 +0000 2009,"ainda bem, eu vou morar no céu",True,6351,Jrodrigues1954
4,passaporte vacinal,PedroVa18482025,Tue Feb 15 23:59:29 +0000 2022,RT @Jrodrigues1954: REJEITOU:\nA Assembleia Le...,0,1387,10727,Wed Oct 28 15:51:59 +0000 2020,"Varginha, Brasil",True,6351,Jrodrigues1954


In case you didn't want to run the code above again (that took 1 hour, at least), you can get the csv file and turn the data back into a DataFrame object (via Pandas).

In [ ]:
# get values of specific columns on the chosen csv. file
df = pd.read_csv('tweets_dataframe.csv', usecols= ['keyword','user','retweeted_from','retweet_fav_count','retweet_count','favorite_count'])